# Computer Vision 2022 1st Lab - Part 3: Εφαρμογές σε Ταιριασμα και Κατηγοριοποίηση Εικόνων με Χρήση Τοπικών Περιγραφητών στα Σημεία Ενδιαφέροντος<br>

Ηλιόπουλος Γεώργιος: 03118815 <br>
Σερλής Αναστάσιος Εμανουήλ - 03118125

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import cv22_lab1_part3_utils as p3

import math
from matplotlib import rcParams
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from scipy.spatial.distance import cdist

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
import sys
from matplotlib.patches import Circle

def disk_strel(n):
    '''
        Return a structural element, which is a disk of radius n.
    '''
    r = int(np.round(n))
    d = 2*r+1
    x = np.arange(d) - r
    y = np.arange(d) - r
    x, y = np.meshgrid(x,y)
    strel = x**2 + y**2 <= r**2
    return strel.astype(np.uint8)

def interest_points_visualization(I_, kp_data_, ax=None):
    '''
    Plot keypoints chosen by detectos on image.
    Args:
        I_: Image (if colored, make sure it is in RGB and not BGR).
        kp_data_: Nx3 array, as described in assignment.
        ax: Matplotlib axis to plot on (if None, a new Axes object is created).
    Returns:
        ax: Matplotlib axis where the image was plotted.
    '''
    try:
        I = np.array(I_)
        kp_data = np.array(kp_data_)
    except:
        print('Conversion to numpy arrays failed, check if the inputs (image and keypoints) are in the required format.')
        exit(2)

    try:
        assert(len(I.shape) == 2 or (len(I.shape) == 3 and I.shape[2] == 3))
    except AssertionError as e:
        print('interest_points_visualization: Image must be either a 2D matrix or a 3D matrix with the last dimension having size equal to 3.', file=sys.stderr)
        exit(2)

    try:
        assert(len(kp_data.shape) == 2 and kp_data.shape[1] == 3)
    except AssertionError as e:
        print('interest_points_visualization: kp_data must be a 2D matrix with 3 columns.', file=sys.stderr)
        exit(2)

    if ax is None:
        _, ax = plt.subplots()

    ax.set_aspect('equal')
    ax.imshow(I)
    ax.tick_params(bottom=False, left=False, labelbottom=False, labelleft=False)

    for i in range(len(kp_data)):
        x, y, sigma = kp_data[i]
        circ = Circle((x, y), 3*sigma, edgecolor='r', fill=False, linewidth=1)
        ax.add_patch(circ)

    return ax

kern = np.array([ #kern is B
    [0,1,0],
    [1,1,1],
    [0,1,0]
], dtype=np.uint8)

In [3]:
def corner_detect(img,sigma=2,r=2.5,k=0.05,theta_corn=0.005):
    
    img_gray=img
    
    n = int(2 * np.ceil(3 * sigma) + 1)
    gauss1D_s = cv2.getGaussianKernel(n, sigma) # Column vector
    gauss2D_s = gauss1D_s @ gauss1D_s.T # Symmetric gaussian kernel
    img_smooth = cv2.filter2D(img_gray, -1, gauss2D_s)

    Is_x,Is_y=np.gradient(img_smooth)

    n=int(2*np.ceil(3*r)+1) 
    gauss1D_r = cv2.getGaussianKernel(n, r) # Column vector
    gauss2D_r = gauss1D_r @ gauss1D_r.T # Symmetric gaussian kernel

    J1= cv2.filter2D(Is_x*Is_x, -1, gauss2D_r) 
    J2= cv2.filter2D(Is_x*Is_y, -1, gauss2D_r)
    J3= cv2.filter2D(Is_y*Is_y, -1, gauss2D_r)

    #2.1.2
    lambda_plus=0.5*(J1+J3+np.sqrt((J1-J3)**2+4*(J2**2)))
    lambda_minus=0.5*(J1+J3-np.sqrt((J1-J3)**2+4*(J2**2)))
    R=lambda_plus*lambda_minus-k*((lambda_plus+lambda_minus)**2)


    #2.1.3
    R=lambda_plus*lambda_minus-k*((lambda_plus+lambda_minus)**2)

    ns = np.ceil(3*sigma)*2+1
    B_sq = disk_strel(ns)
    Cond1 = (R==cv2.dilate(R,B_sq))

    th=theta_corn*np.max(R)
    Cond2 = (R > th)
    final = Cond1 & Cond2

    kp_data=[]
    vertical_len, horizontal_len = np.shape(final)
    for i in range(vertical_len):
        for j in range(horizontal_len):
            if(final[i][j]):
                kp_data.append([j, i, sigma])

    return np.array(kp_data,dtype='object')

In [4]:
def multi_corner_detect(img,sigma=2,r=2.5,k=0.05,theta_corn=0.005,s=1.5,N=4):
    
    img_gray=img
    
    sigmas=np.zeros(N)
    rs=np.zeros(N)
    stupid=[[0]]
    LoG=[]
    for i in range(N):
        sigmas[i]=(s**i)*sigma
        rs[i]=(r**i)*r


    R=[]
    for i in range(N):
        m = int(2*np.ceil(3*rs[i])+1)
        n = int(2 * np.ceil(3 * sigmas[i]) + 1)
        gauss1D_s = cv2.getGaussianKernel(n, sigmas[i]) # Column vector
        gauss2D_s = gauss1D_s @ gauss1D_s.T # Symmetric gaussian kernel
        gauss1D_r = cv2.getGaussianKernel(m, rs[i]) # Column vector  
        gauss2D_r = gauss1D_r @ gauss1D_r.T # Symmetric gaussian kernel
        img_smooth = cv2.filter2D(img_gray, -1, gauss2D_s)
    
    
    
        Lx,Ly=np.gradient(img_smooth)
        Lxx,Lxy=np.gradient(Lx)
        Lyx,Lyy=np.gradient(Ly)
        LoG.append(np.array((((sigmas[i])**2)*abs(Lxx+Lyy))))
    
        Is_x,Is_y=np.gradient(img_smooth)
    
        J1=(cv2.filter2D(Is_x*Is_x, -1, gauss2D_r)) 
        J2=(cv2.filter2D(Is_x*Is_y, -1, gauss2D_r)) 
        J3=(cv2.filter2D(Is_y*Is_y, -1, gauss2D_r)) 
    
    
        lambda_plus=(0.5*(J1+J3+np.sqrt((J1-J3)**2+4*(J2**2))))
        lambda_minus=(0.5*(J1+J3-np.sqrt((J1-J3)**2+4*(J2**2))))
        R.append(np.array(lambda_plus*lambda_minus-k*((lambda_plus+lambda_minus)**2)))


                 
    finals=[]
    for i in range(N):
        ns = np.ceil(3*sigmas[i])*2+1
        B_sq = disk_strel(ns)
        Cond1 = ( R[i]==cv2.dilate(R[i],B_sq))
        th=theta_corn*np.max(R[i])
        Cond2 = (R[i] > th)
        if i==0: 
            Cond3 = (LoG[i]>LoG[i+1])
        if i==N-1:
            Cond3 = (LoG[i]>LoG[i-1])
        else:
            Cond3 = ((LoG[i]>LoG[i-1]) & (LoG[i]>LoG[i+1]))
        temp_boolean = Cond1 & Cond2 & Cond3
        finals.append(temp_boolean)
    
    kp_data=[]
     
    for i in range(N):  
        vertical_len, horizontal_len=np.shape(finals[i])
        for j in range(vertical_len):
            for k in range(horizontal_len):
                if(finals[i][j][k]):
                    kp_data.append([k, j, sigmas[i]])
    
    if np.shape(kp_data) == (0,0,0):
        kp_data.append([1][2][3])


    return np.array(kp_data,dtype='object')

In [5]:
def blob_detect(img,sigma=2,r=2.5,k=0.05,theta_corn=0.005):

    img_gray=img
    
    n = int(2 * np.ceil(3 * sigma) + 1)
    gauss1D_s = cv2.getGaussianKernel(n, sigma) # Column vector
    gauss2D_s = gauss1D_s @ gauss1D_s.T # Symmetric gaussian kernel
    img_smooth = cv2.filter2D(img_gray, -1, gauss2D_s)
 
    Lx,Ly=np.gradient(img_smooth) #Ls
    Lxx,Lxy=np.gradient(Lx)
    Lyx,Lyy=np.gradient(Ly)

    R=Lxx*Lyy-(Lxy*Lxy) #det


    #2.3.2
    ns = np.ceil(3*sigma)*2+1 #arxika cond1+cond2
    B_sq = disk_strel(ns)
    Cond1 = (R==cv2.dilate(R,B_sq))

    th=theta_corn*np.max(R)
    Cond2 = (R > th)
    final = Cond1 & Cond2

    kp_data=[]
    vertical_len, horizontal_len = np.shape(final)
    for i in range(vertical_len):
        for j in range(horizontal_len):
            if(final[i][j]):
                kp_data.append([j, i, sigma])

    return np.array(kp_data,dtype='object')
    


In [6]:
def multi_blob_detect(img,sigma=2,r=2.5,k=0.05,theta_corn=0.005,s=1.5,N=4):
     
    img_gray=img
    
    sigmas=np.zeros(N)
    rs=np.zeros(N)
    stupid=[[0]]
    LoG=[]
    for i in range(N):
        sigmas[i]=(s**i)*sigma
        rs[i]=(r**i)*r


    R=[]
    for i in range(N):
        n = int(2 * np.ceil(3 * sigmas[i]) + 1)
        gauss1D_s = cv2.getGaussianKernel(n, sigmas[i]) # Column vector
        gauss2D_s = gauss1D_s @ gauss1D_s.T # Symmetric gaussian kernel
        img_smooth = cv2.filter2D(img_gray, -1, gauss2D_s)
        Lx,Ly=np.gradient(img_smooth)
        Lxx,Lxy=np.gradient(Lx)
        Lyx,Lyy=np.gradient(Ly)
        LoG.append(np.array((((sigmas[i])**2)*abs(Lxx+Lyy))))
        R.append(np.array(Lxx*Lyy-Lxy*Lyx))


    finals=[]
    for i in range(N):
        ns = np.ceil(3*sigmas[i])*2+1
        B_sq = disk_strel(ns)
        Cond1 = ( R[i]==cv2.dilate(R[i],B_sq))
        th=theta_corn*np.max(R[i])
        Cond2 = (R[i] > th)
        if i==0: 
            Cond3 = (LoG[i]>LoG[i+1])
        if i==N-1:
            Cond3 = (LoG[i]>LoG[i-1])
        else:
            Cond3 = ((LoG[i]>LoG[i-1]) & (LoG[i]>LoG[i+1]))
            temp_boolean = Cond1 & Cond2 & Cond3
        finals.append(temp_boolean)
    
    
    kp_data=[] 
    for i in range(N):  
        vertical_len, horizontal_len=np.shape(finals[i])
        for j in range(vertical_len):
            for k in range(horizontal_len):
                if(finals[i][j][k]):
                    kp_data.append([k, j, sigmas[i]])
    
    return np.array(kp_data,dtype='object')

In [7]:
def box_filters(img,sigma=2,r=2.5,k=0.05,theta_corn=0.005):
    img_gray = img.astype(np.float)/255
    
    n = int (2*np.ceil(3*sigma)+1)
        
    gaussian = cv2.getGaussianKernel(n,sigma) 
    gauss2D = gaussian @ gaussian.T
    I = cv2.filter2D(img_gray, -1, gauss2D)
    
    olokl_x=np.cumsum(img_gray,axis=0)
    olokl_y=np.cumsum(img_gray,axis=1)
    olokl=np.cumsum(olokl_x,axis=1)

    n = int(2 * np.ceil(3 * sigma) + 1)
    n1=int(4*np.floor(n/6)+1) 
    n2=int(2*np.floor(n/6)+1)  
    n3=int((n-n1)/2)
    n4=int((n-n2)/2)
    n5=int((n-1)/2)  
    n6=int((n-2*n2)/3)
    n7=int((n1-1)/2)
    n8=int((n2-1)/2)
    n9=int((n6-1)/2)
    
    thres0=5*int(n5)
    thres=0
    
    Lxx = np.array([[0 for j in range(np.shape(olokl)[1])] for i in range(np.shape(olokl)[0])])
    Lyy = np.array([[0 for j in range(np.shape(olokl)[1])] for i in range(np.shape(olokl)[0])])
    Lxy = np.array([[0 for j in range(np.shape(olokl)[1])] for i in range(np.shape(olokl)[0])])
    
    olokl_ext=np.array([[0 for j in range(np.shape(olokl)[1]+thres0)] for i in range(np.shape(olokl)[0]+thres0)])

    for i, row in enumerate(olokl):
        for j, col in enumerate(row):
            if( (i>=n5) and (i<np.shape(olokl)[0]+n5) and (j>=n5) and (j<np.shape(olokl)[1] +n5)):
                olokl_ext[i][j]=olokl[i][j]
                
    olokl_ext_x=np.array([[0 for j in range(np.shape(olokl)[1]+thres0)] for i in range(np.shape(olokl)[0]+thres0)])

    for i, row in enumerate(olokl):
        for j, col in enumerate(row):
            if( (i>=n5) and (i<np.shape(olokl)[0]+n5) and (j>=n5) and (j<np.shape(olokl)[1] +n5 )):
                   olokl_ext_x[i][j]=olokl_x[i][j]
                
    olokl_ext_y=np.array([[0 for j in range(np.shape(olokl)[1]+thres0)] for i in range(np.shape(olokl)[0]+thres0)])

    for i, row in enumerate(olokl):
        for j, col in enumerate(row):
            if( (i>=n5) and (i<np.shape(olokl)[0]+n5) and (j>=n5) and (j<np.shape(olokl)[1]+n5 )):
                   olokl_ext_y[i][j]=olokl_y[i][j]
                    
    for i, row1 in enumerate(olokl_ext_x):
        for j, col1 in enumerate(row1):
            if ( (i>=n5) and (i<n5+np.shape(olokl)[0]-thres) and (j>=n5) and (j<n5+np.shape(olokl)[1]-thres)): 
                LA2=(-2)*olokl_ext[i-n7][j-n8]
                LC2=(-2)*olokl_ext[i+n7][j+n8]
                LB2=(-2)*olokl_ext[i-n7][j+n8]
                LD2=(-2)*olokl_ext[i+n7][j-n8]
                LX2=LA2+LC2-LB2-LD2
                LA1=olokl_ext[i-n7][j-n8-(2*n8+1)]
                LC1=olokl_ext[i+n7][j+n8-(2*n8+1)]
                LB1=olokl_ext[i-n7][j+n8-(2*n8+1)]
                LD1=olokl_ext[i+n7][j-n8-(2*n8+1)]
                LX1=LA1+LC1-LB1-LD1
                LA3=olokl_ext[i-n7][j-n8+(2*n8+1)]
                LD3=olokl_ext[i+n7][j-n8+(2*n8+1)]
                LC3=olokl_ext[i+n7][j+n8+(2*n8+1)]
                LB3=olokl_ext[i-n7][j+n8+(2*n8+1)]
                LX3=LA3+LC3-LB3-LD3
                Lxx[i-n5][j-n5]=LX1+LX2+LX3
            #if(Lxx[i-n5][j-n5]>100):Lxx[i-n5][j-n5]=100
            #if(Lxx[i-n5][j-n5]<-200):Lxx[i-n5][j-n5]=-200

            
    for i, row1 in enumerate(olokl_ext_y):
        for j, col1 in enumerate(row1):
            if ( (i>=n5) and (i<n5+np.shape(olokl)[0]-thres) and (j>=n5) and (j<n5+np.shape(olokl)[1]-thres)): 
                LA2=(-2)*olokl_ext[i-n8][j-n7]
                LC2=(-2)*olokl_ext[i+n8][j+n7]
                LB2=(-2)*olokl_ext[i-n8][j+n7]
                LD2=(-2)*olokl_ext[i+n8][j-n7]
                LY2=LA2+LC2-LB2-LD2
                LA1=olokl_ext[i-n8-(2*n8+1)][j-n7]
                LC1=olokl_ext[i+n8-(2*n8+1)][j+n7]
                LB1=olokl_ext[i-n8-(2*n8+1)][j+n7]
                LD1=olokl_ext[i+n8-(2*n8+1)][j-n7]
                LY1=LA1+LC1-LB1-LD1
                LA3=olokl_ext[i-n8+(2*n8+1)][j-n7]
                LB3=olokl_ext[i-n8+(2*n8+1)][j+n7]
                LD3=olokl_ext[i+n8+(2*n8+1)][j-n7]
                LC3=olokl_ext[i+n8+(2*n8+1)][j+n7]
                LY3=LA3+LC3-LB3-LD3
                Lyy[i-n5][j-n5]=LY1+LY2+LY3   
            #if(Lyy[i-n5][j-n5]>100):Lyy[i-n5][j-n5]=100
            #if(Lyy[i-n5][j-n5]<-200):Lyy[i-n5][j-n5]=-200


    if(n9==0):n9=1
    

    for i, row1 in enumerate(olokl_ext):
        for j, col1 in enumerate(row1):
            if ( (i>=n5) and (i<n5+np.shape(olokl)[0]-thres) and (j>=n5) and (j<n5+np.shape(olokl)[1]-thres)): 
                LA1=olokl_ext[i-(n9+n8)-n8][j-(n9+n8)-n8]
                LB1=olokl_ext[i-(n9+n8)-n8][j-(n9+n8)+n8]
                LC1=olokl_ext[i-(n9+n8)+n8][j-(n9+n8)+n8]
                LD1=olokl_ext[i-(n9+n8)+n8][j-(n9+n8)-n8]
                L1=LA1+LC1-LD1-LB1
            
                LA2=(-1)*olokl_ext[i-(n9+n8)-n8][j+(n9+n8)-n8]
                LB2=(-1)*olokl_ext[i-(n9+n8)-n8][j+(n9+n8)+n8]
                LC2=(-1)*olokl_ext[i-(n9+n8)+n8][j+(n9+n8)+n8]
                LD2=(-1)*olokl_ext[i-(n9+n8)+n8][j+(n9+n8)-n8]
                L2=LA2+LC2-LD2-LB2
            
                LA3=(-1)*olokl_ext[i+(n9+n8)-n8][j-(n9+n8)-n8]
                LB3=(-1)*olokl_ext[i+(n9+n8)-n8][j-(n9+n8)+n8]
                LC3=(-1)*olokl_ext[i+(n9+n8)+n8][j-(n9+n8)+n8]
                LD3=(-1)*olokl_ext[i+(n9+n8)+n8][j-(n9+n8)-n8]
                L3=LA3+LC3-LD3-LB3
            
                LA4=olokl_ext[i+(n9+n8)-n8][j+(n9+n8)-n8]
                LB4=olokl_ext[i+(n9+n8)-n8][j+(n9+n8)+n8]
                LC4=olokl_ext[i+(n9+n8)+n8][j+(n9+n8)+n8]
                LD4=olokl_ext[i+(n9+n8)+n8][j+(n9+n8)-n8]
                L4=LA4+LC4-LD4-LB4
            
                Lxy[i-n5][j-n5]=L1+L2+L3+L4
            #if(Lxy[i-n5][j-n5]>100):Lxy[i-n5][j-n5]=100
            #if(Lxy[i-n5][j-n5]<-200):Lxy[i-n5][j-n5]=-200
            
    R=Lxx*Lyy-(0.9*Lxy)**2
    R = (R - R.min())/R.max() # turn to binary
    ns = np.ceil(3*sigma)*2+1 #arxika cond1+cond2
    B_sq = disk_strel(ns)
    Cond1 = (R==cv2.dilate(R,B_sq))
    

    th=theta_corn*np.max(R) 
    Cond2 = (R > th)
    final = Cond1 & Cond2


    kp_data=[]
    vertical_len, horizontal_len = np.shape(final)
    for i in range(vertical_len):
        for j in range(horizontal_len):
            if(final[i][j]):
                kp_data.append([j, i, sigma])

    return np.array(kp_data,dtype='object')

In [8]:
def multi_box_filter(img,sigma=2,r=2.5,k=0.05,theta_corn=0.005,s=1.5,N=4):
    img_gray = img.astype(np.float)/255
    
    olokl_x=np.cumsum(img_gray,axis=0)
    olokl_y=np.cumsum(img_gray,axis=1)
    olokl=np.cumsum(olokl_x,axis=1)
    
    sigmas=np.zeros(N)
    rs=np.zeros(N)
    stupid=[[0]]
    LoG=[]
    for i in range(N):
        sigmas[i]=(s**i)*sigma
        rs[i]=(r**i)*r


    R=[]
    for k in range(N):
        n = int(2 * np.ceil(3 * sigmas[k]) + 1)
        n1=int(4*np.floor(n/6)+1)
        n2=int(2*np.floor(n/6)+1)
        n3=int((n-n1)/2)
        n4=int((n-n2)/2)
        n5=int((n-1)/2)
        n6=int((n-2*n2)/3)
        n7=int((n1-1)/2)
        n8=int((n2-1)/2)
        n9=int((n6-1)/2)
        
        thres0=5*int(n5)
        thres=0
    
        Lxx = np.array([[0 for j in range(np.shape(olokl)[1])] for i in range(np.shape(olokl)[0])])
        Lyy = np.array([[0 for j in range(np.shape(olokl)[1])] for i in range(np.shape(olokl)[0])])
        Lxy = np.array([[0 for j in range(np.shape(olokl)[1])] for i in range(np.shape(olokl)[0])])
    
        olokl_ext=np.array([[0 for j in range(np.shape(olokl)[1]+thres0)] for i in range(np.shape(olokl)[0]+thres0)])

        for i, row in enumerate(olokl):
            for j, col in enumerate(row):
                if( (i>=n5) and (i<np.shape(olokl)[0]+n5) and (j>=n5) and (j<np.shape(olokl)[1] +n5)):
                    olokl_ext[i][j]=olokl[i][j]
                
        olokl_ext_x=np.array([[0 for j in range(np.shape(olokl)[1]+thres0)] for i in range(np.shape(olokl)[0]+thres0)])

        for i, row in enumerate(olokl):
            for j, col in enumerate(row):
                if( (i>=n5) and (i<np.shape(olokl)[0]+n5) and (j>=n5) and (j<np.shape(olokl)[1] +n5 )):
                       olokl_ext_x[i][j]=olokl_x[i][j]
                
        olokl_ext_y=np.array([[0 for j in range(np.shape(olokl)[1]+thres0)] for i in range(np.shape(olokl)[0]+thres0)])

        for i, row in enumerate(olokl):
            for j, col in enumerate(row):
                if( (i>=n5) and (i<np.shape(olokl)[0]+n5) and (j>=n5) and (j<np.shape(olokl)[1]+n5 )):
                       olokl_ext_y[i][j]=olokl_y[i][j]
                    
        for i, row1 in enumerate(olokl_ext_x):
            for j, col1 in enumerate(row1):
                if ( (i>=n5) and (i<n5+np.shape(olokl)[0]-thres) and (j>=n5) and (j<n5+np.shape(olokl)[1]-thres)): 
                    LA2=(-2)*olokl_ext[i-n7][j-n8]
                    LC2=(-2)*olokl_ext[i+n7][j+n8]
                    LB2=(-2)*olokl_ext[i-n7][j+n8]
                    LD2=(-2)*olokl_ext[i+n7][j-n8]
                    LX2=LA2+LC2-LB2-LD2
                    LA1=olokl_ext[i-n7][j-n8-(2*n8+1)]
                    LC1=olokl_ext[i+n7][j+n8-(2*n8+1)]
                    LB1=olokl_ext[i-n7][j+n8-(2*n8+1)]
                    LD1=olokl_ext[i+n7][j-n8-(2*n8+1)]
                    LX1=LA1+LC1-LB1-LD1
                    LA3=olokl_ext[i-n7][j-n8+(2*n8+1)]
                    LD3=olokl_ext[i+n7][j-n8+(2*n8+1)]
                    LC3=olokl_ext[i+n7][j+n8+(2*n8+1)]
                    LB3=olokl_ext[i-n7][j+n8+(2*n8+1)]
                    LX3=LA3+LC3-LB3-LD3
                    Lxx[i-n5][j-n5]=LX1+LX2+LX3
                    
        for i, row1 in enumerate(olokl_ext_y):
            for j, col1 in enumerate(row1):
                if ( (i>=n5) and (i<n5+np.shape(olokl)[0]-thres) and (j>=n5) and (j<n5+np.shape(olokl)[1]-thres)): 
                    LA2=(-2)*olokl_ext[i-n8][j-n7]
                    LC2=(-2)*olokl_ext[i+n8][j+n7]
                    LB2=(-2)*olokl_ext[i-n8][j+n7]
                    LD2=(-2)*olokl_ext[i+n8][j-n7]
                    LY2=LA2+LC2-LB2-LD2
                    LA1=olokl_ext[i-n8-(2*n8+1)][j-n7]
                    LC1=olokl_ext[i+n8-(2*n8+1)][j+n7]
                    LB1=olokl_ext[i-n8-(2*n8+1)][j+n7]
                    LD1=olokl_ext[i+n8-(2*n8+1)][j-n7]
                    LY1=LA1+LC1-LB1-LD1
                    LA3=olokl_ext[i-n8+(2*n8+1)][j-n7]
                    LB3=olokl_ext[i-n8+(2*n8+1)][j+n7]
                    LD3=olokl_ext[i+n8+(2*n8+1)][j-n7]
                    LC3=olokl_ext[i+n8+(2*n8+1)][j+n7]
                    LY3=LA3+LC3-LB3-LD3
                    Lyy[i-n5][j-n5]=LY1+LY2+LY3  
        
        if(n9==0):n9=1
    

        for i, row1 in enumerate(olokl_ext):
            for j, col1 in enumerate(row1):
                if ( (i>=n5) and (i<n5+np.shape(olokl)[0]-thres) and (j>=n5) and (j<n5+np.shape(olokl)[1]-thres)): 
                    LA1=olokl_ext[i-(n9+n8)-n8][j-(n9+n8)-n8]
                    LB1=olokl_ext[i-(n9+n8)-n8][j-(n9+n8)+n8]
                    LC1=olokl_ext[i-(n9+n8)+n8][j-(n9+n8)+n8]
                    LD1=olokl_ext[i-(n9+n8)+n8][j-(n9+n8)-n8]
                    L1=LA1+LC1-LD1-LB1
            
                    LA2=(-1)*olokl_ext[i-(n9+n8)-n8][j+(n9+n8)-n8]
                    LB2=(-1)*olokl_ext[i-(n9+n8)-n8][j+(n9+n8)+n8]
                    LC2=(-1)*olokl_ext[i-(n9+n8)+n8][j+(n9+n8)+n8]
                    LD2=(-1)*olokl_ext[i-(n9+n8)+n8][j+(n9+n8)-n8]
                    L2=LA2+LC2-LD2-LB2
            
                    LA3=(-1)*olokl_ext[i+(n9+n8)-n8][j-(n9+n8)-n8]
                    LB3=(-1)*olokl_ext[i+(n9+n8)-n8][j-(n9+n8)+n8]
                    LC3=(-1)*olokl_ext[i+(n9+n8)+n8][j-(n9+n8)+n8]
                    LD3=(-1)*olokl_ext[i+(n9+n8)+n8][j-(n9+n8)-n8]
                    L3=LA3+LC3-LD3-LB3
            
                    LA4=olokl_ext[i+(n9+n8)-n8][j+(n9+n8)-n8]
                    LB4=olokl_ext[i+(n9+n8)-n8][j+(n9+n8)+n8]
                    LC4=olokl_ext[i+(n9+n8)+n8][j+(n9+n8)+n8]
                    LD4=olokl_ext[i+(n9+n8)+n8][j+(n9+n8)-n8]
                    L4=LA4+LC4-LD4-LB4
            
                    Lxy[i-n5][j-n5]=L1+L2+L3+L4
        LoG.append(np.array((((sigmas[k])**2)*abs(Lxx+Lyy))))
        R.append(np.array(Lxx*Lyy-(0.9*Lxy)**2))
    
   
    finals=[]
    for i in range(N):
        R[i]= (R[i] - R[i].min())/R[i].max() # turn to binary
        ns = np.ceil(3*sigmas[i])*2+1
        B_sq = disk_strel(ns)
        Cond1 = ( R[i]==cv2.dilate(R[i],B_sq))
        th=theta_corn*np.max(R[i])
        Cond2 = (R[i] > th)
        if i==0: 
            Cond3 = (LoG[i]>LoG[i+1])
        if i==N-1:
            Cond3 = (LoG[i]>LoG[i-1])
        else:
            Cond3 = ((LoG[i]>LoG[i-1]) & (LoG[i]>LoG[i+1]))
            temp_boolean = Cond1 & Cond2 & Cond3
        finals.append(temp_boolean)



    kp_data=[]
     
    for i in range(N):  
        vertical_len, horizontal_len=np.shape(finals[i])
        for j in range(vertical_len):
            for k in range(horizontal_len):
                if(finals[i][j][k]):
                    kp_data.append([k, j, sigmas[i]])
                    
    return np.array(kp_data,dtype='object')

In [22]:
def evaluationn(detector,descriptor):
    detect_fun = lambda img: detector(img)

    desc_fun = lambda img, kp: descriptor(img,kp)

    s_errors, t_errors = p3.matching_evaluation(detect_fun, desc_fun)
    
    for i in range(3): 
        print('Avg. Scale Error for Image ',i+1,': {:.3f}'.format(s_errors[i]))
        print('Avg. Theta Error for Image ',i+1,': {:.3f}'.format(t_errors[i]))

In [23]:
detectors = [corner_detect, multi_corner_detect, blob_detect, multi_blob_detect, box_filters, multi_box_filter]
descriptors = [p3.featuresSURF, p3.featuresHOG]

(2,)


In [24]:
for i in range(2):
    print("Descriptor:", '\033[1m' +  descriptors[i].__name__ + '\033[0m')
    for j in range(6):
        print("Detector:", '\033[1m' +  detectors[j].__name__ + '\033[0m')
        evaluationn(detectors[j], descriptors[i])

Descriptor: featuresSURF
Detector: corner_detect
Avg. Scale Error for Image  1 : 0.003
Avg. Theta Error for Image  1 : 1.968
Avg. Scale Error for Image  2 : 0.002
Avg. Theta Error for Image  2 : 0.318
Avg. Scale Error for Image  3 : 0.097
Avg. Theta Error for Image  3 : 12.909
Detector: multi_corner_detect
Avg. Scale Error for Image  1 : 0.002
Avg. Theta Error for Image  1 : 0.134
Avg. Scale Error for Image  2 : 0.140
Avg. Theta Error for Image  2 : 18.328
Avg. Scale Error for Image  3 : 0.082
Avg. Theta Error for Image  3 : 10.015
Detector: blob_detect
Avg. Scale Error for Image  1 : 0.027
Avg. Theta Error for Image  1 : 7.759
Avg. Scale Error for Image  2 : 0.010
Avg. Theta Error for Image  2 : 0.229
Avg. Scale Error for Image  3 : 0.001
Avg. Theta Error for Image  3 : 0.054
Detector: multi_blob_detect
Avg. Scale Error for Image  1 : 0.002
Avg. Theta Error for Image  1 : 0.100
Avg. Scale Error for Image  2 : 0.002
Avg. Theta Error for Image  2 : 0.058
Avg. Scale Error for Image  3 : 

In [13]:
def classs(detector, descriptor):
    detect_fun = lambda img: detector(img)

    desc_fun = lambda img, kp: descriptor(img,kp)
    

 # Extract features from the provided dataset.
    feats = p3.FeatureExtraction(detect_fun, desc_fun, saveFile='corner_detect_SURF')

    accs=[]

    for k in range(5):
        # Split into a training set and a test set.
        data_train, label_train, data_test, label_test = p3.createTrainTest(feats, k)

        # Perform Kmeans to find centroids for clusters.
        BOF_tr, BOF_ts = p3.BagOfWords(data_train, data_test)

        # Train an svm on the training set and make predictions on the test set
        acc, preds, probas = p3.svm(BOF_tr, label_train, BOF_ts, label_test)
        accs.append(acc)
    
    return(100.0*np.mean(accs))

In [14]:
detectors = [multi_corner_detect, multi_blob_detect, multi_box_filter]
descriptors = [p3.featuresSURF, p3.featuresHOG]

In [15]:
for i in range(2):
    print("Descriptor:", '\033[1m' +  descriptors[i].__name__ + '\033[0m')
    for j in range(3):
        print("Detector:", '\033[1m' +  detectors[j].__name__ + '\033[0m')
        result1 = classs(corner_detect, p3.featuresHOG)
        print('Mean accuracy is: {:.3f}%'.format(result1))

Descriptor: featuresSURF
Detector: multi_corner_detect
Time for feature extraction: 61.296
Mean accuracy is: 62.759%
Detector: multi_blob_detect
Time for feature extraction: 57.677
Mean accuracy is: 61.517%
Detector: multi_box_filter
Time for feature extraction: 57.156
Mean accuracy is: 62.897%
Descriptor: featuresHOG
Detector: multi_corner_detect
Time for feature extraction: 54.672
Mean accuracy is: 64.000%
Detector: multi_blob_detect
Time for feature extraction: 58.804
Mean accuracy is: 60.690%
Detector: multi_box_filter
Time for feature extraction: 67.557
Mean accuracy is: 62.621%


In [19]:
def histogram_maker(image, centroids):
    #compute pairwise euclidean distance for each image from each centroid
    distance = cdist(image, centroids)
    
    # keep as label the centroid of minimum distance
    labels = np.zeros(distance.shape[0])
    for i in range(labels.shape[0]):
        labels[i] = np.argwhere(distance[i] == np.min(distance[i]))
    
    # Create Histogram
    histogram, bins = np.histogram(labels, np.arange(0,centroids.shape[0]))
    
    # Normalize
    normalize = np.linalg.norm(histogram)
    histogram = histogram / normalize
    
    return histogram

def BOVW_Manual(data_tr, data_ts):
    # Define useful variables
    nc = 500;
    sp = 0.5;

    data_descr = np.vstack(data_tr) #concatenate all of them

    # Get a random subset of data_descriptors.
    length = np.ceil(np.shape(data_descr)[0] * sp)
    i = np.random.choice(np.shape(data_descr)[0], int(length))
    sample = data_descr[i]

    # Perform k-means algorithm
    kmeans = KMeans(n_clusters=nc).fit(sample)
    idx, centroids = kmeans.labels_, kmeans.cluster_centers_ 
    
    # Create Bag of Words for Train Test using Histograms
    BOF_tr = []
    for image in (data_tr):
        BOF_tr.append(histogram_maker(image, centroids).tolist())
    BOF_tr = np.stack(BOF_tr, axis=0)
    
    # Repeat procedure for Test Set
    BOF_ts = []
    for image in (data_ts):
        BOF_ts.append(histogram_maker(image, centroids).tolist())
    BOF_ts = np.stack(BOF_ts, axis=0)
    
    return BOF_tr, BOF_ts



### Bonus Υλοποίηση (3.2.3)

In [20]:
def classs_manual(detector, descriptor):
    detect_fun = lambda img: detector(img)

    desc_fun = lambda img, kp: descriptor(img,kp)

 # Extract features from the provided dataset.
    feats = p3.FeatureExtraction(detect_fun, desc_fun, saveFile='newnew')

    accs=[]

    for k in range(5):
        # Split into a training set and a test set.
        data_train, label_train, data_test, label_test = p3.createTrainTest(feats, k)

        # Perform Kmeans to find centroids for clusters.
        BOF_tr, BOF_ts = BOVW_Manual(data_train, data_test)

        # Train an svm on the training set and make predictions on the test set
        acc, preds, probas = p3.svm(BOF_tr, label_train, BOF_ts, label_test)
        accs.append(acc)
    
    return(100.0*np.mean(accs))


In [21]:
for i in range(2):
    print("Descriptor:", '\033[1m' +  descriptors[i].__name__ + '\033[0m')
    for j in range(3):
        print("Detector:", '\033[1m' +  detectors[j].__name__ + '\033[0m')
        result1 = classs_manual(corner_detect, p3.featuresHOG)
        print('Mean accuracy is: {:.3f}%'.format(result1))

Descriptor: featuresSURF
Detector: multi_corner_detect
Time for feature extraction: 76.399
Mean accuracy is: 61.793%
Detector: multi_blob_detect
Time for feature extraction: 75.000
Mean accuracy is: 60.966%
Detector: multi_box_filter
Time for feature extraction: 76.401
Mean accuracy is: 61.103%
Descriptor: featuresHOG
Detector: multi_corner_detect
Time for feature extraction: 73.109
Mean accuracy is: 63.172%
Detector: multi_blob_detect
Time for feature extraction: 73.876
Mean accuracy is: 60.138%
Detector: multi_box_filter
Time for feature extraction: 72.909
Mean accuracy is: 61.241%
